In [62]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/joaquin/Documents/GitHub/skforecast'

In [63]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.utils.utils import align_series_and_exog_multiseries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.datasets import fetch_dataset


%load_ext pyinstrument
%load_ext line_profiler

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [64]:
n = 1000
series = [pd.Series(np.random.normal(n), index=pd.date_range(start='2000-01-01', periods=n, freq='h'), name=f'series_{i}') for i in range(500)]
series = pd.concat(series, axis=1).melt(var_name='series_id', value_name='value', ignore_index=False)
series = series.rename_axis('datetime')
series = series.reset_index()
series

,datetime,series_id,value
0,2000-01-01 00:00:00,series_0,998.189779
1,2000-01-01 01:00:00,series_0,998.189779
2,2000-01-01 02:00:00,series_0,998.189779
3,2000-01-01 03:00:00,series_0,998.189779
4,2000-01-01 04:00:00,series_0,998.189779
...,...,...,...
499995,2000-02-11 11:00:00,series_499,997.932077
499996,2000-02-11 12:00:00,series_499,997.932077
499997,2000-02-11 13:00:00,series_499,997.932077
499998,2000-02-11 14:00:00,series_499,997.932077


In [65]:
# From long format to long format with multiple and frequancy
# ==================================================================================================
series = series.groupby('series_id').apply(lambda x: x.set_index('datetime').asfreq('h'), include_groups=False)
series

value
series_id datetime                        
series_0  2000-01-01 00:00:00   998.189779
          2000-01-01 01:00:00   998.189779
          2000-01-01 02:00:00   998.189779
          2000-01-01 03:00:00   998.189779
          2000-01-01 04:00:00   998.189779
...                                    ...
series_99 2000-02-11 11:00:00  1000.817620
          2000-02-11 12:00:00  1000.817620
          2000-02-11 13:00:00  1000.817620
          2000-02-11 14:00:00  1000.817620
          2000-02-11 15:00:00  1000.817620

[500000 rows x 1 columns]

In [66]:
import pandas as pd

def check_frequency_multiseries(series: pd.DataFrame):

    assert isinstance(series.index, pd.MultiIndex)
    assert series.index.nlevels == 2

    series_ids = series.index.get_level_values(0)
    timestamps = series.index.get_level_values(1)
    time_diffs = timestamps.to_series().groupby(series_ids).diff().dropna()
    unique_deltas = np.unique(time_diffs)

    
    if len(unique_deltas) != 1:
        raise ValueError(
            "The time differences within each series are not consistent. "
            "Expected a single unique time difference, but found: "
            f"{time_diffs.drop_duplicates().astype(str).to_numpy()}"
        )


check_frequency_multiseries(series)

In [72]:
%%timeit

series_ids = series.index.get_level_values(0)
timestamps = series.index.get_level_values(1)
freqs = timestamps.to_series().groupby(series_ids).diff().dropna()
# time_diffs.drop_duplicates().astype(str).to_numpy()
freqs

50.7 ms ± 644 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [85]:
%%timeit
freqs = series.reset_index().groupby('series_id')['datetime'].diff().dropna()
freqs

62.1 ms ± 705 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [73]:
%%timeit
unique_ids = series.index.levels[0]
indexes_freq = [
    series.loc[series_id].index.freq for series_id in unique_ids
]
indexes_freq

83.3 ms ± 1.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
%%timeit

series_no_index = series.reset_index()
time_diffs = series_no_index.groupby('series_id')['datetime'].diff().dropna()
time_diffs.drop_duplicates().astype(str).to_numpy()

58.3 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
# Concat series multiindex with exog multiindex equal for all series
# ==================================================================================================
exog = pd.DataFrame({
    'exog_1': [1, 2, 3, 4, 5, 6, 7, 8],
    'exog_2': [10, 20, 30, 40, 50, 60, 70, 80]
}, index=pd.date_range(start='2000-01-01', periods=8, freq='D'))
exog = exog.rename_axis('datetime')
display(exog)

# merge series and exog
series_exog = pd.merge(series, exog, left_index=True, right_index=True, how='left')
series_exog

,exog_1,exog_2
datetime,,
2000-01-01,1,10
2000-01-02,2,20
2000-01-03,3,30
2000-01-04,4,40
2000-01-05,5,50
2000-01-06,6,60
2000-01-07,7,70
2000-01-08,8,80


value  exog_1  exog_2
series_id datetime                         
series_1  2000-01-01      1     1.0    10.0
          2000-01-02      2     2.0    20.0
          2000-01-03      3     3.0    30.0
          2000-01-04      4     4.0    40.0
          2000-01-05      5     5.0    50.0
          2000-01-06      6     6.0    60.0
          2000-01-07      7     7.0    70.0
          2000-01-08      8     8.0    80.0
          2000-01-09      9     NaN     NaN
          2000-01-10     10     NaN     NaN
series_2  2000-01-01     20     1.0    10.0
          2000-01-02     30     2.0    20.0
          2000-01-03     40     3.0    30.0
          2000-01-04     50     4.0    40.0
          2000-01-05     60     5.0    50.0
          2000-01-06     70     6.0    60.0
          2000-01-07     80     7.0    70.0
          2000-01-08     90     8.0    80.0
          2000-01-09    100     NaN     NaN
          2000-01-10    110     NaN     NaN

In [8]:
# Concat series multiindex with exog multiindex diferent exog for each series
# ==================================================================================================
exog_1 = pd.DataFrame({
    'series_id' : 'series_1',
    'exog_1': [1, 2, 3, 4, 5, 6, 7, 8, 9 , 10],
    'exog_2': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}, index=pd.date_range(start='2000-01-01', periods=10, freq='D'))
exog_2 = pd.DataFrame({
    'series_id' : 'series_2',
    'exog_1': [50, 60, 70, 80, 90, 100, 110, 120, 130, 140],
    #'exog_2': [500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400]
}, index=pd.date_range(start='2000-01-01', periods=10, freq='D'))
exog = pd.concat([exog_1, exog_2])
exog = exog.set_index(['series_id', exog.index])
exog.index.names = ['series_id', 'datetime']
exog

# merge series and exog
series_exog = pd.merge(series, exog, left_index=True, right_index=True, how='left')
series_exog

value  exog_1  exog_2
series_id datetime                         
series_1  2000-01-01      1       1    10.0
          2000-01-02      2       2    20.0
          2000-01-03      3       3    30.0
          2000-01-04      4       4    40.0
          2000-01-05      5       5    50.0
          2000-01-06      6       6    60.0
          2000-01-07      7       7    70.0
          2000-01-08      8       8    80.0
          2000-01-09      9       9    90.0
          2000-01-10     10      10   100.0
series_2  2000-01-01     20      50     NaN
          2000-01-02     30      60     NaN
          2000-01-03     40      70     NaN
          2000-01-04     50      80     NaN
          2000-01-05     60      90     NaN
          2000-01-06     70     100     NaN
          2000-01-07     80     110     NaN
          2000-01-08     90     120     NaN
          2000-01-09    100     130     NaN
          2000-01-10    110     140     NaN

In [10]:
# From long format multiindex to dict
# ==============================================================================
series_ids = series_exog.index.get_level_values(0).unique()
series_dict = {
    sid: series_exog.loc[sid] for sid in series_ids
}
series_dict

{'series_1':             value  exog_1  exog_2
 datetime                         
 2000-01-01      1       1    10.0
 2000-01-02      2       2    20.0
 2000-01-03      3       3    30.0
 2000-01-04      4       4    40.0
 2000-01-05      5       5    50.0
 2000-01-06      6       6    60.0
 2000-01-07      7       7    70.0
 2000-01-08      8       8    80.0
 2000-01-09      9       9    90.0
 2000-01-10     10      10   100.0,
 'series_2':             value  exog_1  exog_2
 datetime                         
 2000-01-01     20      50     NaN
 2000-01-02     30      60     NaN
 2000-01-03     40      70     NaN
 2000-01-04     50      80     NaN
 2000-01-05     60      90     NaN
 2000-01-06     70     100     NaN
 2000-01-07     80     110     NaN
 2000-01-08     90     120     NaN
 2000-01-09    100     130     NaN
 2000-01-10    110     140     NaN}